In [1]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter


import pandas as pd
import numpy as np

import sklearn


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import seaborn as sns

import pydot



In [2]:
data_train = pd.read_csv('/content/full_hin.csv')
data_train.head()

Unnamed: 0                                               Text  label
0           0  Bollywood film dekhne ke samay logic ghar mein...      0
1           1                                   Chutiya movie...      0
2           2       Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने      0
3           3  @Pankaj Chauhan abey tum jse aadmiyo ko ye lgt...      0
4           4                 bro house of card ka review karona      0

In [3]:
data_test = pd.read_csv('/content/trac2_hin_dev.csv')

In [4]:
data_test.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [5]:
NAG_test = data_test[(data_test['Sub-task A']=='NAG')]
OAG_test= data_test[(data_test['Sub-task A']=='CAG')]
CAG_test = data_test[(data_test['Sub-task A']=='OAG')]

In [6]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG_test["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG_test["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG_test["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG_test.head()

Text  Sub-task A
1   Bhaiya shaadi mein zaroor aana movie ka plot j...           0
2   Section 375 hai kya??? .... Ye to batate kam s...           0
4              Maine itni kam dislike kbhi nhii dekhi           0
7   जिश देश मे super30 जैसी फ़िल्म फ्लॉप ओर कबीर सि...           0
11            Bhai movie review 5 se 7 mins Bano yaar           0

In [7]:
data_test = pd.concat([NAG_test,
                  OAG_test,
                  CAG_test
                 ],axis=0)

data_test.reset_index(inplace=True)


In [8]:

data_test = data_test.rename(columns = {'Sub-task A': 'label'}, inplace = False)

In [9]:
X = data_train.Text.astype(str)
Y = data_train.label
#le = LabelEncoder()
#Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)

In [10]:
X_train,X_valid,y_train,y_valid = train_test_split(X,Y,test_size=0.15)

In [11]:
num_words = 50000

tokenizer = Tokenizer(num_words=num_words,oov_token="unk")
tokenizer.fit_on_texts(X_train)


print(str(tokenizer.texts_to_sequences(['xyz how are you'])))

[[88, 3694, 202, 222]]


In [12]:

x_train = np.array( tokenizer.texts_to_sequences(X_train) )
x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
x_test  = np.array( tokenizer.texts_to_sequences(data_test['Text'].tolist()) )


In [13]:

x_train = pad_sequences(x_train, padding='post', maxlen=150)
x_valid = pad_sequences(x_valid, padding='post', maxlen=150)
x_test = pad_sequences(x_test, padding='post', maxlen=150)

In [14]:
train_labels = np.asarray(y_train )
valid_labels = np.asarray( y_valid)

test_labels = np.asarray(data_test['label'].tolist())


In [15]:

max_features =50000
embedding_dim =16
sequence_length = 150


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features, 64, input_length=sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 30
history_bi = model.fit(x_train,train_labels, epochs=num_epochs, validation_data=(x_valid, valid_labels))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           3200000   
_________________________________________________________________
bidirectional (Bidirectional (None, 150, 128)          66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 3,311,489
Trainable params: 3,311,489
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
201/201 [==============================] - 71s 319ms/step - loss: 0.0000e+00 - accuracy: 0.3698 - val

In [16]:

x_test = np.array( tokenizer.texts_to_sequences(data_test['Text'].tolist()) )
x_test = pad_sequences(x_test, padding = 'post', maxlen=150)


In [17]:
print("Generate predictions for all samples")
predictions = model.predict(x_test)

Generate predictions for all samples


In [18]:

data_test['pred']= predictions

In [19]:

labels = [0,1,2]
    
print(classification_report(data_test['label'].tolist(),data_test['pred'].tolist(),labels=labels))

              precision    recall  f1-score   support

           0       0.70      0.60      0.64       578
           1       0.22      0.53      0.31       211
           2       0.00      0.00      0.00       208

    accuracy                           0.46       997
   macro avg       0.31      0.38      0.32       997
weighted avg       0.45      0.46      0.44       997

